In [1]:
import pandas as pd
import re # Регулярные выражения.
#import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
#import pymorphy2 # Морфологический анализатор.
#import datetime # Новости будем перебирать по дате.
#from collections import Counter # Не считать же частоты самим.
#import math # Корень квадратный.
import numpy

In [2]:
train = pd.read_csv('train.csv', sep = '\t', index_col='id') #memory error

In [3]:
# удалим html коды
def _remove_all_attrs(soup):
    text = soup
    soup = BeautifulSoup(text, "lxml")
    return soup.get_text()

# Пробуем нормировать сам текст
def clean_text(text):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols  
    #text = " ".join(ma.parse(str(word))[0].normal_form for word in text.split())
    text = ' '.join(word for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")

    return text

In [4]:
train['description'] = train['description'].apply(_remove_all_attrs)

In [5]:
train['description_full'] = train['name'] + ' ' + train['description']

In [6]:
train['description_full'] =  train['description_full'].apply(clean_text)

In [7]:
y = train.target

In [10]:
#train_work = train

In [12]:
#train_work = train_work.drop('description', axis = 1)

In [14]:
#train_work = train_work.drop('name', axis = 1)

In [15]:
#train_work = train_work.drop('target', axis = 1)

In [47]:
#train_work.head()

,description_full
id,
0,заведующий отделом секцией магазин york уручье...
1,наладчик станков манипуляторов обязанности раб...
2,разработчик криптограф требования опыт програм...
3,фрезеровщик условия работу вахтовым методом ка...
4,мерчендайзер продавец консультант компания пал...


In [78]:
del train_work

In [9]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
model  = Pipeline([
    ('vect', TfidfVectorizer(stop_words='english', 
                             ngram_range=(1,3),
                             max_df=0.95, min_df=5)),
    ('clf', LogisticRegression()),
])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train['description_full'], y, test_size=0.33, random_state=42)

In [13]:
model.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
 ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [14]:
test = pd.read_csv('test.csv', sep='\t', encoding='utf8')

In [15]:
test['description'] = test['description'].apply(_remove_all_attrs)

In [16]:
test['description_full'] = test['name'] + ' ' + test['description']

In [17]:
test['description_full'] =  test['description_full'].apply(clean_text)

In [25]:
test_predict = model.predict_proba(X=test['description_full'])

In [28]:
test_predict[:,1]

array([ 0.95301283,  0.9932517 ,  0.99402377, ...,  0.46136071,
        0.02390364,  0.00913287])

In [20]:
test.head()

,id,name,description,description_full
0,200000,Дизайнер-консультант мебели,"Обязанности: Работа с клиентом в салоне,выезд...",дизайнер консультант мебели обязанности работа...
1,200001,Продавец-консультант (ТЦ на Пушкина),Обязанности: ∙ консультирование покупателей по...,продавец консультант пушкина обязанности консу...
2,200002,Менеджер по продажам,Торговый Дом «Форт» это ведущая компания Петер...,менеджер продажам торговый «форт» ведущая комп...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,Требуются продавцы консультанты в магазин женс...,продавец консультант магазин одежды волна ново...
4,200004,Специалист по охране труда,Обязанности: осуществление контроля по соблю...,специалист охране труда обязанности осуществле...


In [29]:
submission = pd.DataFrame({"id":test["id"],
                           "target":test_predict[:,1]})

In [30]:
submission.head()

,id,target
0,200000,0.953013
1,200001,0.993252
2,200002,0.994024
3,200003,0.967539
4,200004,0.012123


In [31]:
submission.to_csv("hw_06_submit.csv", 
                  index=False)